In [2]:
import sqlite3
import json

In [ ]:
captures = """CREATE TABLE captures (
capture TEXT NOT NULL,
versie INTEGER NOT NULL,
oid INTEGER NOT NULL,
lot INTEGER NOT NULL,
verdund BOOLEAN NOT NULL,
grootte BOOLEAN NOT NULL,
PRIMARY KEY (capture, versie, lot)
)
"""

In [ ]:
pakketten = """CREATE TABLE pakketten (
pakket TEXT NOT NULL,
versie INTEGER NOT NULL,
grootte BOOLEAN,
PRIMARY KEY (pakket, versie)
)
"""

In [ ]:
panels = """CREATE TABLE panels (
panels TEXT NOT NULL,
versie INTEGER NOT NULL,
grootte BOOLEAN,
PRIMARY KEY (panels, versie)
)
"""

In [14]:
capgenes = dict()

conn = sqlite3.connect('../ngstargets/varia/capinfo.sqlite')
c = conn.cursor()
c.execute('SELECT capture, genen FROM genes')

for output in c.fetchall():
    capgenes[output[0]] = json.loads(output[1])


In [16]:
capgenes.keys()

dict_keys(['SCDv6', 'DLv2', 'NEUROv4', 'AGPPVMv1', 'COWSOv1', 'MBSv4', 'BHv3', 'CHCv1', 'LYMPHv4', 'CMv15', 'PCOv2'])